In [1]:
import json
from mention import *
from collections import Counter
from math import log
from gensim.matutils import argsort
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
import numpy as np
import random
import sys
from random import randint
from data_load import *
%load_ext cython

In [65]:
%%cython
import numpy as np

cimport numpy as np
from random import randint
import sys
import cython
cdef extern from "math.h":
    double sqrt(double m)
import math
from libc.stdlib cimport malloc, free

from libc.math cimport exp
from libc.math cimport log
from gensim.matutils import argsort

from libc.string cimport memset
import random
# scipy <= 0.15

import scipy.linalg.blas as fblas
ctypedef np.float32_t REAL_t
cdef int ONE = 1


REAL = np.float32
cdef extern from "/Users/mayk/working/figer/baseline/PLE/Model/warp/voidptr.h":
    void* PyCObject_AsVoidPtr(object obj)
DEF MAX_SENTENCE_LEN = 10000
ctypedef void (*scopy_ptr) (const int *N, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef void (*saxpy_ptr) (const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil
ctypedef float (*sdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*dsdot_ptr) (const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil
ctypedef double (*snrm2_ptr) (const int *N, const float *X, const int *incX) nogil
ctypedef void (*sscal_ptr) (const int *N, const float *alpha, const float *X, const int *incX) nogil


cdef scopy_ptr scopy = <scopy_ptr>PyCObject_AsVoidPtr(fblas.scopy._cpointer)  # y = x
cdef saxpy_ptr saxpy=<saxpy_ptr>PyCObject_AsVoidPtr(fblas.saxpy._cpointer)  # y += alpha * x
cdef sdot_ptr sdot=<sdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # float = dot(x, y)
cdef dsdot_ptr dsdot=<dsdot_ptr>PyCObject_AsVoidPtr(fblas.sdot._cpointer)  # double = dot(x, y)
cdef snrm2_ptr snrm2=<snrm2_ptr>PyCObject_AsVoidPtr(fblas.snrm2._cpointer)  # sqrt(x^2)
cdef sscal_ptr sscal=<sscal_ptr>PyCObject_AsVoidPtr(fblas.sscal._cpointer) # x = alpha * x
DEF EXP_TABLE_SIZE = 10000
DEF MAX_EXP = 50

cdef REAL_t[EXP_TABLE_SIZE] EXP_TABLE
cdef REAL_t[EXP_TABLE_SIZE] LOG_TABLE

cdef REAL_t ONEF = <REAL_t>1.0

# for when fblas.sdot returns a double
cdef REAL_t our_dot_double(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>dsdot(N, X, incX, Y, incY)

# for when fblas.sdot returns a float
cdef REAL_t our_dot_float(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    return <REAL_t>sdot(N, X, incX, Y, incY)

# for when no blas available
cdef REAL_t our_dot_noblas(const int *N, const float *X, const int *incX, const float *Y, const int *incY) nogil:
    # not a true full dot()-implementation: just enough for our cases
    cdef int i
    cdef REAL_t a
    a = <REAL_t>0.0
    for i from 0 <= i < 50 by 1:
        a += X[i] * Y[i]
    return a

# for when no blas available
cdef void our_saxpy_noblas(const int *N, const float *alpha, const float *X, const int *incX, float *Y, const int *incY) nogil:
    cdef int i
    for i from 0 <= i < N[0] by 1:
        Y[i * (incY[0])] = (alpha[0]) * X[i * (incX[0])] + Y[i * (incY[0])]
cdef REAL_t cvdot(vec1,vec2,size):
    cdef int csize = size
    f= dsdot(&csize,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
def csaxpy(vec1,vec2,alpha,size):
    cdef int csize = size
    cdef float calpha = alpha
    f= our_saxpy_noblas(&csize,&calpha,<REAL_t *>(np.PyArray_DATA(vec1)),&ONE,<REAL_t *>(np.PyArray_DATA(vec2)),&ONE)
    return f
cdef REAL_t crank(int k):
    cdef REAL_t loss = 0.
    cdef int i = 1
    for i in range(1,k+1):
        loss += ONEF/i
    return loss
cdef REAL_t vsum(REAL_t *vec,int *size):
    cdef int i
    cdef REAL_t product
    product = <REAL_t>0.0
    for i from 0 <= i < size[0] by 1:
        product += vec[i] **2
    return sqrt(product)
def cnorm(vec):
    cdef int size
    size  = len(vec)
    return vsum(<REAL_t *>(np.PyArray_DATA(vec)),&size)
def init():
    for i in range(EXP_TABLE_SIZE):
        EXP_TABLE[i] = <REAL_t>exp((i / <REAL_t>EXP_TABLE_SIZE * 2 - 1) * MAX_EXP)
        EXP_TABLE[i] = <REAL_t>(EXP_TABLE[i] / (EXP_TABLE[i] + 1))
#init()


def ctrain(A,B,C,insts,size,lr,gradient,it,Verbose=False):
    cdef float error
    next_random = 1
    error = 0.
    for i,inst in enumerate(insts):
        err,next_random =gradient(A,B,C,inst,size,next_random,lr=lr)
        error += err
        if i % 1000 ==0 and Verbose:
            sys.stdout.write("\rIteration %d " % (it)+ "trained {0:.0f}%".format(float(i)*100/len(insts))+" Loss:{0:.2f}".format(error))
            sys.stdout.flush()
    if Verbose:
        sys.stdout.write("\n")
    return error

cdef void divide(REAL_t *vec, const float *alpha, const int *size):
    cdef int i
    for i from 0 <= i < size[0] by 1:
        vec[i] = vec[i]/alpha[0]
def cdivide(vec,alpha):
    cdef int size
    size  = len(vec)
    cdef float r = alpha
    divide(<REAL_t *>(np.PyArray_DATA(vec)),&r,&size)

    
cdef REAL_t matdot(vec1,mat,size):
    cdef REAL_t product = 0.
    for i in range(len(mat)):
        product+= cvdot(vec1,mat[i],size)
    return product

def warp_gradient(A,B,C,inst,size,next_random,lr=0.01):
    #print B
    #print B[0]-B[9]
    cdef unsigned long long  c_next_random = next_random
    dA = np.zeros(size,dtype=REAL)
    dB = np.zeros(B.shape,dtype=REAL)
    x = np.sum(A[inst.features],axis=0)
    cdef REAL_t error = 0.
    cdef REAL_t clr = lr
    cdef int N,n_sample 
    cdef int neg_num = len(inst.negative_labels)
    cdef REAL_t norm
    cdef int cSize = size
    cdef REAL_t floats
    M = len(inst.sparse_labels)
    for i,l in enumerate(inst.sparse_labels):
        y = np.dot(C[l],B)
        s1= cvdot(x,y,size)#cvdot(x,B[l],cSize)#
        N=1
        n_sample  = -1
        for k in range(neg_num):
            c_next_random = random_int32(&c_next_random)
            nl = inst.negative_labels[c_next_random%neg_num]#randint(0,neg_num-1)]
            y_neg = np.dot(C[nl],B)
            s2 = cvdot(x,y_neg,size)#cvdot(x,B[nl],size)#
            
            if s1 - s2<1:
                n_sample = nl
                N = k+1
                break
        if n_sample!=-1:

            L = crank(len(inst.negative_labels)/N)#*(crank(M/(ranks[i]+1)))
            negL = -L
            error += (1+s2-s1)*L

            csaxpy(y-y_neg,dA,L,size)
            for i in range(len(dB)):
                csaxpy(C[l][i]*x,dB[i],L,size)
                csaxpy(C[n_sample][i]*x,dB[i],-L,size)
            #print dB[l][0]
    for f in inst.features:
        csaxpy(dA,A[f],clr,size)
        norm = cnorm(A[f])
        if norm >1:
            cdivide(A[f],norm)

    for i in range(len(B)):
        csaxpy(dB[i],B[i],clr,size)

        #B[i] += lr*dB[i]
        norm =  cnorm(B[i])
        if norm >1:
            cdivide(B[i],norm)
            #B[i] /=norm
    return error,c_next_random
def save_to_text(matrix,output):
    shape = matrix.shape
    with open(output,'wb') as out:
        out.write("%d %d\n" % (shape))
        for row in matrix:
            x = " ".join(map(lambda x:"{0:.5}".format(x),row))
            out.write(x+"\n")

cdef inline unsigned long long random_int32(unsigned long long *next_random) nogil:
    next_random[0] = (next_random[0] * <unsigned long long>25214903917ULL + 11) & 281474976710655ULL
    return next_random[0]
def crand(sed):
    cdef unsigned long long csed = sed
    return random_int32(&csed)
def save2bin(mat,dct,fn):
    n,d  = mat.shape
    with open(fn,'w') as out:
        out.write("%d %d\n" % (n,d))
        for i in range(n):
            text = " ".join(map(str,mat[i]))
            out.write("%s %s\n" %(dct[i],text))
def normalize(mat):
    for v in mat:
        norm = np.linalg.norm(v)
        if norm >=1:
            v /= norm
            

In [3]:
stop = set([ ln.rstrip() for ln in open('stop.list')])

In [5]:
def total_count(d):
    return reduce(lambda x,y:x+y,d.values(),0)
def lambda_p(l,e,labels,entities,label_entity,t_l):
    if "%s_%s" % (l,e) not in label_entity: return -1.
    c = log(t_l)
    return (log(labels[l]) + log(entities[e]) - 2*c)/(log(label_entity["%s_%s" % (l,e)])-c) -1
def proto(l,es,labels,entities,label_entity,t_l,topn=10):
    scores = [lambda_p(l,e,labels,entities,label_entity,t_l) for e in es]
    return [es[i] for i in argsort(scores,topn=topn,reverse=True)]

In [11]:

with open('/Volumes/My Passport/RR_backup/data/dependency_embedding/deps.words','rb') as vectors, open('/Volumes/My Passport/RR_backup/data/dependency_embedding/shortlist','wb') as out:
    short = [ln for ln in vectors if ln.split()[0] in words.token2id]
    out.write("%d %d\n" % (len(short),300))
    for ln in short:
        out.write(ln)
words = Dictionary([e.split() for e in es])

In [ ]:

model = Word2Vec.load_word2vec_format('../../bow2.words',binary=True)
t_l = total_count(labels)
es = [e for e in entities]
with open('labels.bin','w') as lout:
    lout.write("%d %d\n" % (len(labels),300))
    for l in labels:
        protoes = proto(l,es,labels,entities,label_entity,t_l,topn=60)
        print l,protoes
        avg = np.zeros(300,dtype=np.float32)
        hit = 0
        for w in protoes:
            if w in model.vocab:
                hit+=1
                avg += model[w]
        if hit ==0:
            print "shit"
        avg /= hit
        text =  " ".join(map(str,avg))

        lout.write("%s %s\n" % (l,text))


In [4]:
label_bin = Word2Vec.load_word2vec_format('/Users/mayk/working/figer/baseline/PLE/Model/warp/labels.bin')

In [5]:
in_dir= "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN"
a=MentionData('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_x_new.txt',
              "/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_y.txt",
             in_dir+"/feature.txt",in_dir+"/type.txt")

In [67]:
np.random.seed(2)
size= 50

A= np.random.uniform(-6/np.sqrt(size),6/np.sqrt(size), [len(a.feature2id),size]).astype(np.float32)#np.random.rand(len(a.feature2id),size).astype(np.float32)
C = np.asarray([label_bin[a.id2label[i]] for i in range(len(label_bin.vocab))],dtype=np.float32)
B= np.random.uniform(-6/np.sqrt(size),6/np.sqrt(size), [C.shape[1],size]).astype(np.float32)# np.random.rand(len(a.label2id),size).astype(np.float32)
next_random = 1
normalize(A)
normalize(C)
normalize(B)
for i in range(5): 
    reg_err =0.
    ctrain(A,B,C,a.data,size,0.001,warp_gradient,it=i,Verbose=True)


save2bin(A,a.id2feature,'/Users/mayk/working/figer/baseline/PLE/Results/warp_A.bin')

save2bin(np.dot(C,B),a.id2label,'/Users/mayk/working/figer/baseline/PLE/Results/warp_B.bin')


Iteration 0 trained 100% Loss:209659.22
Iteration 1 trained 100% Loss:138851.02
Iteration 2 trained 100% Loss:112356.00
Iteration 3 trained 100% Loss:96400.90
Iteration 4 trained 100% Loss:84271.61


In [48]:
np.dot(C,B).shape

(47, 300)

In [7]:
size =50
C = np.asarray([label_bin[a.id2label[i]] for i in range(len(label_bin.vocab))])
B= np.random.uniform(-6/np.sqrt(size),6/np.sqrt(size), [300,size]).astype(np.float32)

In [25]:
model = Word2Vec.load_word2vec_format('shortlist',binary=False)
t_l = total_count(labels)
es = [e for e in entities]
with open('labels.bin','w') as lout, open('/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/type_man.txt') as man:
    lout.write("%d %d\n" % (len(labels),300))
    for l in man:
        l= l.rstrip().split()[3]
        if l not in model:print l
        #avg = model[l]
#         hit = 0
#         for w in protoes:
#             if w in model.vocab:
#                 hit+=1
#                 avg += model[w]
#         if hit ==0:
#             print "shit"
#         avg /= hit
#         text =  " ".join(map(str,avg))

#         lout.write("%s %s\n" % (l,text))

location
url
region
facility
attraction
song
drama
religion


10

[(u'/ORGANIZATION/GOVERNMENT', 0.6716739535331726),
 (u'/LOCATION/LAKE_SEA_OCEAN', 0.655185878276825),
 (u'/ORGANIZATION/EDUCATIONAL', 0.6380061507225037),
 (u'/WORK_OF_ART/BOOK', 0.6343929767608643),
 (u'/ORGANIZATION/MUSEUM', 0.6125108003616333),
 (u'/FACILITY/BRIDGE', 0.6027704477310181),
 (u'/ORGANIZATION/CORPORATION', 0.5899884700775146),
 (u'/EVENT/HURRICANE', 0.5854605436325073),
 (u'/WORK_OF_ART/PLAY', 0.5838527083396912),
 (u'/LAW', 0.5786466598510742)]

In [245]:
Word2Vec.lo

u'/WORK_OF_ART/PLAY'